# Demo

Set up spark context and SparkSession

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Load dataset

In [2]:
df = spark.read.format('com.databricks.spark.csv').\
                       options(header='true', \
                       inferschema='true').\
            load("../data/Advertising.csv",header=True);
df.show(5,True)
df.printSchema()
df.describe().show()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|             

Convert the data to dense vector (features and label)

In [3]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# I provide two ways to build the features and labels

# method 1 (good for small feature):
#def transData(row):
#    return Row(label=row["Sales"],
#               features=Vectors.dense([row["TV"],
#                                       row["Radio"],
#                                       row["Newspaper"]]))

# Method 2 (good for large features):
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

transformed= transData(df)
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



Deal With Categorical Variables

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)


+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



Split the data into training and test sets (40% held out for testing)

In [5]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = transformed.randomSplit([0.6, 0.4])
trainingData.show(5)
testData.show(5)

+----------------+-----+
|        features|label|
+----------------+-----+
|  [4.1,11.6,5.7]|  3.2|
| [7.8,38.9,50.6]|  6.6|
|   [8.6,2.1,1.0]|  4.8|
|[11.7,36.9,45.2]|  7.3|
| [17.2,4.1,31.6]|  5.9|
+----------------+-----+
only showing top 5 rows

+---------------+-----+
|       features|label|
+---------------+-----+
| [0.7,39.6,8.7]|  1.6|
| [5.4,29.9,9.4]|  5.3|
|[7.3,28.1,41.4]|  5.5|
| [8.4,27.2,2.1]|  5.7|
|[8.7,48.9,75.0]|  7.2|
+---------------+-----+
only showing top 5 rows



Apply Model:
    
Due to the sparsity within our data, our training sets will often be ill-posed (singular). Applying regularization to the regression has many advantages, including:

Converting ill-posed problems to well-posed by adding additional information via the penalty parameter \lambda
Preventing overfitting
Variable selection and the removal of correlated variables (Glmnet Vignette). The Ridge method shrinks the coefficients of correlated variables while the LASSO method picks one variable and discards the others. The elastic net penalty is a mixture of these two; if variables are correlated in groups then \alpha=0.5 tends to select the groups as in or out. If α is close to 1, the elastic net performs much like the LASSO method and removes any degeneracies and wild behavior caused by extreme correlations.



# Ordinary least squares regression (L2 penalty)
![](./image/ols.png)

In [6]:
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
# lr =LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction", maxIter=100,
# regParam=0.1, elasticNetParam=0.0, tol=1e-6, fitIntercept=True, standardization=True, solver="auto",
# weightCol=None, aggregationDepth=2)

lr =LinearRegression(featuresCol="features", labelCol="label", 
                     predictionCol="prediction", maxIter=100,
                        regParam=0.1, elasticNetParam=0.0, tol=1e-6, fitIntercept=True, 
                     standardization=True, solver="auto",
                     aggregationDepth=2)

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, lr])

model = pipeline.fit(trainingData)

def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
           % Summary.meanSquaredError, ",RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", Total iterations: %i"% Summary.totalIterations)
modelsummary(model.stages[-1])


# Make predictions.
predictions = model.transform(testData)
# Select example rows to display.
predictions.select("features","label","prediction").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

Note: the last rows are the information for Intercept
## -------------------------------------------------
##   Estimate   |   Std.Error | t Values  |  P-value
##   0.042992   0.001614   26.643   0.000000
##   0.195859   0.009646   20.305   0.000000
##  -0.000059   0.006570   -0.009   0.992864
##   3.348187   0.355384    9.421   0.000000
## ---
## Mean squared error:  2.064104 ,RMSE:  1.436699
## Multiple R-squared: 0.926180 , Total iterations: 1
+---------------+-----+------------------+
|       features|label|        prediction|
+---------------+-----+------------------+
| [0.7,39.6,8.7]|  1.6|11.133777564563786|
| [5.4,29.9,9.4]|  5.3|  9.43596687661333|
|[7.3,28.1,41.4]|  5.5| 9.163220782315854|
| [8.4,27.2,2.1]|  5.7| 9.036553054806895|
|[8.7,48.9,75.0]|  7.2|13.295293560104216|
+---------------+-----+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2.01457
r2_score: 0.8345527321449107


## Ridge regression (L1 penalty)

![](./image/ridge.png)

In [7]:
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
# lr =LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction", maxIter=100,
# regParam=0.1, elasticNetParam=0.0, tol=1e-6, fitIntercept=True, standardization=True, solver="auto",
# weightCol=None, aggregationDepth=2)

lr =LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction", 
                    maxIter=100, regParam=0.0, elasticNetParam=0.0, tol=1e-6, 
                fitIntercept=True, standardization=True, solver="auto", aggregationDepth=2)

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, lr])

model = pipeline.fit(trainingData)

def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
           % Summary.meanSquaredError, ",RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", Total iterations: %i"% Summary.totalIterations)
modelsummary(model.stages[-1])


# Make predictions.
predictions = model.transform(testData)
# Select example rows to display.
predictions.select("features","label","prediction").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

Note: the last rows are the information for Intercept
## -------------------------------------------------
##   Estimate   |   Std.Error | t Values  |  P-value
##   0.043722   0.001627   26.873   0.000000
##   0.199257   0.009743   20.452   0.000000
##  -0.001022   0.006633   -0.154   0.877801
##   3.184361   0.356948    8.921   0.000000
## ---
## Mean squared error:  2.057158 ,RMSE:  1.434280
## Multiple R-squared: 0.926428 , Total iterations: 1
+---------------+-----+------------------+
|       features|label|        prediction|
+---------------+-----+------------------+
| [0.7,39.6,8.7]|  1.6| 11.09665927920621|
| [5.4,29.9,9.4]|  5.3| 9.368643175490465|
|[7.3,28.1,41.4]|  5.5|  9.06034419947503|
| [8.4,27.2,2.1]|  5.7| 8.969276849953975|
|[8.7,48.9,75.0]|  7.2|13.231761660623357|
+---------------+-----+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2.00698
r2_score: 0.8357977088685933


##  Elastic net ( the penalty is an L1 + L2 penalty)

![](./image/Elastic.png)

In [8]:
from pyspark.ml.regression import LinearRegression



lr =LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction", 
                     maxIter=100, regParam=0.0, elasticNetParam=0.0, tol=1e-6, 
                     fitIntercept=True, standardization=True, solver="auto",
                     aggregationDepth=2)

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, lr])

model = pipeline.fit(trainingData)

def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
           % Summary.meanSquaredError, ",RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", Total iterations: %i"% Summary.totalIterations)
modelsummary(model.stages[-1])


# Make predictions.
predictions = model.transform(testData)
# Select example rows to display.
predictions.select("features","label","prediction").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

Note: the last rows are the information for Intercept
## -------------------------------------------------
##   Estimate   |   Std.Error | t Values  |  P-value
##   0.043722   0.001627   26.873   0.000000
##   0.199257   0.009743   20.452   0.000000
##  -0.001022   0.006633   -0.154   0.877801
##   3.184361   0.356948    8.921   0.000000
## ---
## Mean squared error:  2.057158 ,RMSE:  1.434280
## Multiple R-squared: 0.926428 , Total iterations: 1
+---------------+-----+------------------+
|       features|label|        prediction|
+---------------+-----+------------------+
| [0.7,39.6,8.7]|  1.6| 11.09665927920621|
| [5.4,29.9,9.4]|  5.3| 9.368643175490465|
|[7.3,28.1,41.4]|  5.5|  9.06034419947503|
| [8.4,27.2,2.1]|  5.7| 8.969276849953975|
|[8.7,48.9,75.0]|  7.2|13.231761660623357|
+---------------+-----+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2.00698
r2_score: 0.8357977088685933
